## 1. 基础工作流构建

In [1]:
# 导入基础工作流组件
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
)

# 定义一个简单的工作流
class MyWorkflow(Workflow):
    @step
    async def my_step(self, ev: StartEvent) -> StopEvent:
        # 在这里执行任务
        return StopEvent(result="你好，世界！")

# 创建并运行工作流
w = MyWorkflow(timeout=10, verbose=False)
result = await w.run()
print(result)  # 输出: 你好，世界！

你好，世界！


这个最小工作流仅包含一个步骤，接收StartEvent并返回StopEvent。在Jupyter环境中，可以直接使用await；在普通Python脚本中，需要使用asyncio进行包装;在jupyter环境中使用asyncio包装可能会报错，这个时候需要导入 nest_asyncio 解决问题。

In [3]:
import nest_asyncio
nest_asyncio.apply()

import asyncio

async def main():
    w = MyWorkflow(timeout=10, verbose=False)
    result = await w.run()
    print(result)

if __name__ == "__main__":
    asyncio.run(main())

你好，世界！


## 2. 自定义事件与多步骤工作流

In [4]:
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
)

# 定义自定义事件
class FirstEvent(Event):
    first_output: str

class SecondEvent(Event):
    second_output: str

# 定义三步工作流
class MyWorkflow(Workflow):
    @step
    async def step_one(self, ev: StartEvent) -> FirstEvent:
        print(ev.first_input)
        return FirstEvent(first_output="第一步完成。")

    @step
    async def step_two(self, ev: FirstEvent) -> SecondEvent:
        print(ev.first_output)
        return SecondEvent(second_output="第二步完成。")

    @step
    async def step_three(self, ev: SecondEvent) -> StopEvent:
        print(ev.second_output)
        return StopEvent(result="工作流完成。")

# 运行工作流
w = MyWorkflow(timeout=10, verbose=False)
result = await w.run(first_input="开始工作流。")
print(result)

开始工作流。
第一步完成。
第二步完成。
工作流完成。


这个示例展示了如何通过自定义事件连接多个步骤，形成一个连贯的工作流程。每个步骤接收特定类型的事件，并返回另一个类型的事件，驱动工作流向前推进。

## 3. 添加分支和循环逻辑

实际应用中的工作流通常需要处理条件分支和循环逻辑。

In [5]:
import random
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
)

# 定义事件
class FirstEvent(Event):
    first_output: str

class SecondEvent(Event):
    second_output: str

class LoopEvent(Event):
    loop_output: str

# 包含循环的工作流
class MyWorkflow(Workflow):
    @step 
    async def step_one(self, ev: StartEvent | LoopEvent) -> FirstEvent | LoopEvent:
        if random.randint(0, 1) == 0:
            print("发生了不好的事情")
            return LoopEvent(loop_output="返回第一步。")
        else:
            print("发生了好事")
            return FirstEvent(first_output="第一步完成。")

    @step
    async def step_two(self, ev: FirstEvent) -> SecondEvent:
        print(ev.first_output)
        return SecondEvent(second_output="第二步完成。")

    @step
    async def step_three(self, ev: SecondEvent) -> StopEvent:
        print(ev.second_output)
        return StopEvent(result="工作流完成。")

# 运行工作流
w = MyWorkflow(timeout=10, verbose=False)
result = await w.run()
print(result)

发生了好事
第一步完成。
第二步完成。
工作流完成。


在这个例子中，step_one可以接收两种类型的事件：StartEvent或LoopEvent，并且可以返回两种类型的事件：FirstEvent或LoopEvent。当返回LoopEvent时，控制流会回到step_one，形成循环。

## 4. 工作流中的分支

In [6]:
import random   
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
)

# 定义分支事件
class BranchA1Event(Event):
    payload: str

class BranchA2Event(Event):
    payload: str

class BranchB1Event(Event):
    payload: str

class BranchB2Event(Event):
    payload: str

# 包含分支的工作流
class BranchWorkflow(Workflow):
    @step
    async def start(self, ev: StartEvent) -> BranchA1Event | BranchB1Event:
        if random.randint(0, 1) == 0:
            print("进入分支A")
            return BranchA1Event(payload="分支A")
        else:
            print("进入分支B")
            return BranchB1Event(payload="分支B")

    @step
    async def step_a1(self, ev: BranchA1Event) -> BranchA2Event:
        print(ev.payload)
        return BranchA2Event(payload=ev.payload)

    @step
    async def step_b1(self, ev: BranchB1Event) -> BranchB2Event:
        print(ev.payload)
        return BranchB2Event(payload=ev.payload)

    @step
    async def step_a2(self, ev: BranchA2Event) -> StopEvent:
        print(ev.payload)
        return StopEvent(result="分支A完成。")

    @step
    async def step_b2(self, ev: BranchB2Event) -> StopEvent:
        print(ev.payload)
        return StopEvent(result="分支B完成。")

# 运行工作流
w = BranchWorkflow(timeout=10, verbose=False)
result = await w.run()
print(result)

进入分支A
分支A
分支A
分支A完成。


在这个例子中，工作流根据随机选择进入不同的执行分支。这种能力在需要条件处理的场景中非常有用。

## 5. 状态管理
在工作流中，经常需要在不同步骤之间共享数据。LlamaIndex提供了Context对象来解决这个问题：


In [7]:
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context,
)

# 定义事件
class SetupEvent(Event):
    query: str

class StepTwoEvent(Event):
    query: str

# 有状态的工作流
class StatefulFlow(Workflow):
    @step
    async def start(self, ctx: Context, ev: StartEvent) -> SetupEvent | StepTwoEvent:
        # 获取上下文中的数据
        db = await ctx.get("some_database", default=None)
        if db is None:
            print("需要加载数据")
            return SetupEvent(query=ev.query)
        return StepTwoEvent(query=ev.query)

    @step
    async def setup(self, ctx: Context, ev: SetupEvent) -> StartEvent:
        # 在上下文中设置数据
        await ctx.set("some_database", [1, 2, 3])
        return StartEvent(query=ev.query)

    @step
    async def step_two(self, ctx: Context, ev: StepTwoEvent) -> StopEvent:
        # 使用上下文中的数据
        print("数据是", await ctx.get("some_database"))
        return StopEvent(result=await ctx.get("some_database"))

# 运行工作流
w = StatefulFlow(timeout=10, verbose=False)
result = await w.run(query="示例查询")
print(result)

需要加载数据
数据是 [1, 2, 3]
[1, 2, 3]


## 6. 并发执行
LlamaIndex工作流支持并发执行多个任务，极大提高处理效率：

In [8]:
import random
import asyncio
from llama_index.core.workflow import (
    StartEvent, StopEvent, Workflow, step, Event, Context
)

# 定义事件
class StepTwoEvent(Event):
    query: str

# 并行工作流
class ParallelFlow(Workflow):
    @step
    async def start(self, ctx: Context, ev: StartEvent) -> StepTwoEvent:
        # 发送多个事件并行执行
        ctx.send_event(StepTwoEvent(query="查询1"))
        ctx.send_event(StepTwoEvent(query="查询2"))
        ctx.send_event(StepTwoEvent(query="查询3"))

    @step(num_workers=4)  # 设置最大并发工作线程数
    async def step_two(self, ctx: Context, ev: StepTwoEvent) -> StopEvent:
        # 模拟慢查询
        print(f"执行慢查询: {ev.query}")
        await asyncio.sleep(random.randint(1, 5))
        return StopEvent(result=ev.query)

# 运行工作流
workflow = ParallelFlow(timeout=10, verbose=True)
result = await workflow.run()
print(f"工作流完成，结果: {result}")

Running step start
Step start produced no event
Running step step_two
执行慢查询: 查询1
Running step step_two
执行慢查询: 查询2
Running step step_two
执行慢查询: 查询3
Step step_two produced event StopEvent
Step step_two produced event StopEvent
工作流完成，结果: 查询1


在这个例子中，step_two使用了@step(num_workers=4)装饰器，允许最多同时运行4个该步骤的实例，处理不同的事件。这大大提高了处理多个查询的效率。

In [ ]:
## 7. 事件收集
在并行执行的场景中，我们可能希望等待所有并行任务完成后再继续。LlamaIndex提供了事件收集机制：

In [9]:
import random
import asyncio
from llama_index.core.workflow import (
    StartEvent, StopEvent, Workflow, step, Event, Context
)

# 定义事件
class StepTwoEvent(Event):
    query: str

class StepThreeEvent(Event):
    result: str

# 并发工作流
class ConcurrentFlow(Workflow):
    @step
    async def start(self, ctx: Context, ev: StartEvent) -> StepTwoEvent:
        # 发送多个事件并行执行
        ctx.send_event(StepTwoEvent(query="查询1"))
        ctx.send_event(StepTwoEvent(query="查询2"))
        ctx.send_event(StepTwoEvent(query="查询3"))

    @step(num_workers=4)
    async def step_two(self, ctx: Context, ev: StepTwoEvent) -> StepThreeEvent:
        # 模拟慢查询
        print(f"执行慢查询: {ev.query}")
        await asyncio.sleep(random.randint(1, 5))
        return StepThreeEvent(result=ev.query)
        
    @step
    async def step_three(self, ctx: Context, ev: StepThreeEvent) -> StopEvent:
        # 等待收集3个事件
        result = ctx.collect_events(ev, [StepThreeEvent] * 3)
        if result is None:
            return None  # 返回None表示继续等待更多事件
        
        # 处理所有查询结果
        print("所有查询结果:", result)
        return StopEvent(result="所有查询已完成")

# 运行工作流
workflow = ConcurrentFlow(timeout=10, verbose=True)
result = await workflow.run()
print(f"工作流最终结果: {result}")

Running step start
Step start produced no event
Running step step_two
执行慢查询: 查询1
Running step step_two
执行慢查询: 查询2
Running step step_two
执行慢查询: 查询3
Step step_two produced event StepThreeEvent
Running step step_three
Step step_three produced no event
Step step_two produced event StepThreeEvent
Step step_two produced event StepThreeEvent
Running step step_three
Step step_three produced no event
Running step step_three
所有查询结果: [StepThreeEvent(result='查询1'), StepThreeEvent(result='查询2'), StepThreeEvent(result='查询3')]
Step step_three produced event StopEvent
工作流最终结果: 所有查询已完成


## 8. 可视化工作流
LlamaIndex提供了工具帮助可视化工作流结构，便于理解和调试：

In [13]:
from llama_index.utils.workflow import draw_all_possible_flows

# 可视化工作流
draw_all_possible_flows(MyWorkflow, filename="workflow_visualization.html")

<class 'NoneType'>
<class '__main__.FirstEvent'>
<class '__main__.LoopEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>
<class '__main__.SecondEvent'>
workflow_visualization.html


## 9. 实际应用示例

数据处理工作流

class DataProcessingWorkflow(Workflow):
    @step
    async def fetch_data(self, ctx: Context, ev: StartEvent) -> DataFetchedEvent:
        # 从API获取数据
        data = await fetch_data_from_api(ev.api_endpoint)
        await ctx.set("raw_data", data)
        return DataFetchedEvent(status="数据获取成功")
    
    @step
    async def process_data(self, ctx: Context, ev: DataFetchedEvent) -> DataProcessedEvent:
        # 处理原始数据
        raw_data = await ctx.get("raw_data")
        processed_data = process_raw_data(raw_data)
        await ctx.set("processed_data", processed_data)
        return DataProcessedEvent(status="数据处理成功")
    
    @step
    async def analyze_data(self, ctx: Context, ev: DataProcessedEvent) -> AnalysisCompleteEvent:
        # 分析处理后的数据
        processed_data = await ctx.get("processed_data")
        analysis_result = analyze_data(processed_data)
        await ctx.set("analysis_result", analysis_result)
        return AnalysisCompleteEvent(status="数据分析完成")
    
    @step
    async def generate_report(self, ctx: Context, ev: AnalysisCompleteEvent) -> StopEvent:
        # 生成分析报告
        analysis_result = await ctx.get("analysis_result")
        report = generate_report_from_analysis(analysis_result)
        return StopEvent(result=report)

### 1. 数据处理和分析工作流
这个工作流模拟了一个数据获取、处理、分析和生成报告的完整流程：

In [14]:
import asyncio
import random
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
import base64
from llama_index.core.workflow import (
    StartEvent, StopEvent, Workflow, step, Event, Context
)

# 定义事件
class DataFetchedEvent(Event):
    status: str

class DataProcessedEvent(Event):
    status: str

class AnalysisCompleteEvent(Event):
    status: str

# 模拟数据获取函数
async def fetch_data_from_api(api_endpoint: str):
    print(f"从{api_endpoint}获取数据...")
    await asyncio.sleep(1)  # 模拟API调用延迟
    
    # 模拟生成数据
    data = {
        'date': pd.date_range(start='2023-01-01', periods=30, freq='D'),
        'sales': [random.randint(100, 1000) for _ in range(30)],
        'visitors': [random.randint(500, 5000) for _ in range(30)]
    }
    return pd.DataFrame(data)

# 数据处理函数
def process_raw_data(raw_data):
    print("处理原始数据...")
    
    # 计算移动平均
    processed_data = raw_data.copy()
    processed_data['sales_ma7'] = processed_data['sales'].rolling(window=7).mean()
    processed_data['visitors_ma7'] = processed_data['visitors'].rolling(window=7).mean()
    
    # 计算转化率
    processed_data['conversion_rate'] = processed_data['sales'] / processed_data['visitors'] * 100
    
    return processed_data

# 数据分析函数
def analyze_data(processed_data):
    print("分析处理后的数据...")
    
    # 基本统计分析
    analysis = {
        'total_sales': processed_data['sales'].sum(),
        'avg_daily_sales': processed_data['sales'].mean(),
        'peak_sales_day': processed_data.loc[processed_data['sales'].idxmax(), 'date'].strftime('%Y-%m-%d'),
        'peak_sales': processed_data['sales'].max(),
        'avg_conversion_rate': processed_data['conversion_rate'].mean(),
        'sales_trend': 'increasing' if processed_data['sales'].iloc[-1] > processed_data['sales'].iloc[0] else 'decreasing',
    }
    
    # 创建销售图表
    plt.figure(figsize=(10, 6))
    plt.plot(processed_data['date'], processed_data['sales'], 'b-', label='每日销售')
    plt.plot(processed_data['date'], processed_data['sales_ma7'], 'r-', label='7日移动平均')
    plt.title('销售趋势分析')
    plt.xlabel('日期')
    plt.ylabel('销售额')
    plt.legend()
    plt.grid(True)
    
    # 将图表转换为base64编码的图片
    buffer = BytesIO()
    plt.savefig(buffer, format='png')
    buffer.seek(0)
    image_png = buffer.getvalue()
    buffer.close()
    plt.close()
    
    graph = base64.b64encode(image_png).decode('utf-8')
    analysis['sales_graph'] = graph
    
    return analysis

# 生成报告函数
def generate_report_from_analysis(analysis):
    print("生成分析报告...")
    
    report = f"""
    # 销售数据分析报告
    
    ## 摘要
    - 总销售额: {analysis['total_sales']}
    - 日均销售额: {round(analysis['avg_daily_sales'], 2)}
    - 销售峰值日: {analysis['peak_sales_day']} (销售额: {analysis['peak_sales']})
    - 平均转化率: {round(analysis['avg_conversion_rate'], 2)}%
    - 销售趋势: {analysis['sales_trend']}
    
    ## 销售趋势图表
    ![销售趋势](data:image/png;base64,{analysis['sales_graph']})
    
    ## 建议
    - {'增加营销投入以维持上升趋势' if analysis['sales_trend'] == 'increasing' else '调整销售策略以扭转下降趋势'}
    - 进一步分析峰值销售日的因素，重复成功经验
    - {'转化率表现良好，继续保持' if analysis['avg_conversion_rate'] > 15 else '需要改进网站体验提高转化率'}
    
    *报告生成日期: {pd.Timestamp.now().strftime('%Y-%m-%d')}*
    """
    
    return report

# 数据处理工作流
class DataProcessingWorkflow(Workflow):
    @step
    async def fetch_data(self, ctx: Context, ev: StartEvent) -> DataFetchedEvent:
        # 从API获取数据
        data = await fetch_data_from_api(ev.api_endpoint)
        await ctx.set("raw_data", data)
        return DataFetchedEvent(status="数据获取成功")
    
    @step
    async def process_data(self, ctx: Context, ev: DataFetchedEvent) -> DataProcessedEvent:
        # 处理原始数据
        raw_data = await ctx.get("raw_data")
        processed_data = process_raw_data(raw_data)
        await ctx.set("processed_data", processed_data)
        return DataProcessedEvent(status="数据处理成功")
    
    @step
    async def analyze_data(self, ctx: Context, ev: DataProcessedEvent) -> AnalysisCompleteEvent:
        # 分析处理后的数据
        processed_data = await ctx.get("processed_data")
        analysis_result = analyze_data(processed_data)
        await ctx.set("analysis_result", analysis_result)
        return AnalysisCompleteEvent(status="数据分析完成")
    
    @step
    async def generate_report(self, ctx: Context, ev: AnalysisCompleteEvent) -> StopEvent:
        # 生成分析报告
        analysis_result = await ctx.get("analysis_result")
        report = generate_report_from_analysis(analysis_result)
        return StopEvent(result=report)

# 运行工作流
async def run_data_workflow():
    workflow = DataProcessingWorkflow(timeout=30, verbose=True)
    result = await workflow.run(api_endpoint="https://api.example.com/sales_data")
    
    # 将报告保存到文件
    with open("sales_analysis_report.md", "w", encoding="utf-8") as f:
        f.write(result)
    
    print("\n工作流执行完成!")
    print(f"报告已保存到 sales_analysis_report.md")
    
    # 打印报告摘要
    print("\n报告摘要:")
    print("\n".join(result.split("\n")[:10]) + "\n...")

# 主函数
if __name__ == "__main__":
    asyncio.run(run_data_workflow())

Running step fetch_data
从https://api.example.com/sales_data获取数据...
Step fetch_data produced event DataFetchedEvent
Running step process_data
处理原始数据...
Step process_data produced event DataProcessedEvent
Running step analyze_data
分析处理后的数据...


C:\Users\Cheng\AppData\Local\Temp\ipykernel_19772\1127546308.py:74: UserWarning: Glyph 38144 (\N{CJK UNIFIED IDEOGRAPH-9500}) missing from font(s) DejaVu Sans.
  plt.savefig(buffer, format='png')
C:\Users\Cheng\AppData\Local\Temp\ipykernel_19772\1127546308.py:74: UserWarning: Glyph 21806 (\N{CJK UNIFIED IDEOGRAPH-552E}) missing from font(s) DejaVu Sans.
  plt.savefig(buffer, format='png')
C:\Users\Cheng\AppData\Local\Temp\ipykernel_19772\1127546308.py:74: UserWarning: Glyph 39069 (\N{CJK UNIFIED IDEOGRAPH-989D}) missing from font(s) DejaVu Sans.
  plt.savefig(buffer, format='png')
C:\Users\Cheng\AppData\Local\Temp\ipykernel_19772\1127546308.py:74: UserWarning: Glyph 36235 (\N{CJK UNIFIED IDEOGRAPH-8D8B}) missing from font(s) DejaVu Sans.
  plt.savefig(buffer, format='png')
C:\Users\Cheng\AppData\Local\Temp\ipykernel_19772\1127546308.py:74: UserWarning: Glyph 21183 (\N{CJK UNIFIED IDEOGRAPH-52BF}) missing from font(s) DejaVu Sans.
  plt.savefig(buffer, format='png')
C:\Users\Cheng\AppDa

Step analyze_data produced event AnalysisCompleteEvent
Running step generate_report
生成分析报告...
Step generate_report produced event StopEvent

工作流执行完成!
报告已保存到 sales_analysis_report.md

报告摘要:

    # 销售数据分析报告
    
    ## 摘要
    - 总销售额: 16852
    - 日均销售额: 561.73
    - 销售峰值日: 2023-01-18 (销售额: 949)
    - 平均转化率: 24.98%
    - 销售趋势: increasing
    
...


这个工作流会:
从模拟API获取销售数据
处理数据(计算移动平均和转化率)
分析数据并生成图表
创建一份包含洞察和可视化图表的Markdown报告

### 2. AI辅助内容生成工作流

这个工作流展示了如何使用LlamaIndex工作流结合OpenAI创建一个自动化内容生成系统:

class ContentGenerationWorkflow(Workflow):
    @step
    async def research_topic(self, ctx: Context, ev: StartEvent) -> ResearchCompleteEvent:
        # 研究主题
        research_results = await research_topic_online(ev.topic)
        await ctx.set("research_results", research_results)
        return ResearchCompleteEvent(status="研究完成")
    
    @step
    async def generate_outline(self, ctx: Context, ev: ResearchCompleteEvent) -> OutlineCompleteEvent:
        # 生成内容大纲
        research_results = await ctx.get("research_results")
        outline = await generate_outline_with_llm(research_results, ev.requirements)
        await ctx.set("outline", outline)
        return OutlineCompleteEvent(status="大纲生成完成")
    
    @step
    async def write_content(self, ctx: Context, ev: OutlineCompleteEvent) -> WritingCompleteEvent:
        # 根据大纲撰写内容
        outline = await ctx.get("outline")
        research_results = await ctx.get("research_results")
        
        # 并行撰写各个部分
        sections = outline.split_into_sections()
        for section in sections:
            ctx.send_event(WriteSectionEvent(section=section))
        
        return WritingCompleteEvent(status="内容撰写启动")
    
    @step(num_workers=5)
    async def write_section(self, ctx: Context, ev: WriteSectionEvent) -> SectionCompleteEvent:
        # 撰写单个部分
        research_results = await ctx.get("research_results")
        section_content = await write_section_with_llm(ev.section, research_results)
        
        # 存储部分内容
        sections = await ctx.get("completed_sections", default=[])
        sections.append({"section": ev.section, "content": section_content})
        await ctx.set("completed_sections", sections)
        
        return SectionCompleteEvent(section=ev.section)
    
    @step
    async def compile_content(self, ctx: Context, ev: SectionCompleteEvent) -> StopEvent:
        # 收集所有部分并编译完整内容
        outline = await ctx.get("outline")
        sections_needed = len(outline.split_into_sections())
        
        # 收集所有部分
        result = ctx.collect_events(ev, [SectionCompleteEvent] * sections_needed)
        if result is None:
            return None
        
        # 所有部分都已完成，编译最终内容
        completed_sections = await ctx.get("completed_sections")
        final_content = compile_sections_into_document(completed_sections)
        
        return StopEvent(result=final_content)

In [16]:
import asyncio
import os
import time
from dotenv import load_dotenv
from llama_index.llms.openai import OpenAI
from llama_index.core.workflow import (
    StartEvent, StopEvent, Workflow, step, Event, Context
)
from llama_index.tools.tavily_research import TavilyToolSpec

# 加载环境变量
load_dotenv()

# 确保有必要的API密钥
if not os.getenv("OPENAI_API_KEY"):
    print("警告: 缺少OPENAI_API_KEY环境变量。请在.env文件中设置。")
    os.environ["OPENAI_API_KEY"] = input("请输入你的OpenAI API密钥: ")

if not os.getenv("TAVILY_API_KEY"):
    print("警告: 缺少TAVILY_API_KEY环境变量。请在.env文件中设置。")
    os.environ["TAVILY_API_KEY"] = input("请输入你的Tavily API密钥: ")

# 初始化LLM
llm = OpenAI(model="gpt-3.5-turbo")  # 改为使用更广泛支持的模型

# 初始化Tavily搜索工具
tavily_tool_spec = TavilyToolSpec(api_key=os.getenv("TAVILY_API_KEY"))
search_tool = tavily_tool_spec.to_tool_list()[0]

# 定义事件
class ResearchCompleteEvent(Event):
    status: str

class OutlineCompleteEvent(Event):
    status: str

class WriteSectionEvent(Event):
    section: str
    section_number: int

class SectionCompleteEvent(Event):
    section: str
    section_number: int
    content: str

class WritingCompleteEvent(Event):
    status: str

# 研究主题函数
async def research_topic_online(topic):
    print(f"研究主题: {topic}")
    
    try:
        # 使用Tavily搜索工具获取信息
        search_results = await search_tool.acall(query=f"{topic} 最新信息 关键点")
        
        # 如果结果是字符串，则直接使用
        if isinstance(search_results, str):
            return search_results
        
        # 提取文档文本
        documents = []
        for doc in search_results:
            if hasattr(doc, 'text'):
                documents.append(doc.text)
            elif hasattr(doc, 'text_resource') and doc.text_resource and hasattr(doc.text_resource, 'text'):
                documents.append(doc.text_resource.text)
            elif isinstance(doc, dict) and 'text' in doc:
                documents.append(doc['text'])
        
        # 如果没有有效结果，提供一个默认消息
        if not documents:
            documents = ["无法找到有关该主题的信息。请尝试使用更常见的搜索词。"]
        
        return "\n\n---\n\n".join(documents)
    except Exception as e:
        print(f"研究过程中出错: {str(e)}")
        return f"研究过程中出错: {str(e)}。使用备用信息继续。\n\n人工智能在医疗领域有诊断辅助、药物研发、医学影像分析等应用。"

# 使用LLM生成大纲
async def generate_outline_with_llm(research_results, requirements):
    print("生成内容大纲...")
    
    prompt = f"""
    基于以下研究材料，为主题"{requirements['topic']}"创建一个详细的内容大纲。
    
    要求:
    - 文章类型: {requirements['content_type']}
    - 目标受众: {requirements['audience']}
    - 字数要求: {requirements['word_count']}字左右
    - 风格: {requirements['style']}
    
    请创建一个包含引言、正文(3-5个主要部分)和结论的详细大纲。
    每个部分应该有一个清晰的标题和简短的描述(1-2句)，说明该部分将包含什么内容。
    
    研究材料:
    {research_results[:2000]}  # 限制字符数以避免超出上下文长度
    
    输出格式:
    # 引言
    [引言描述]
    
    # [第一部分标题]
    [第一部分描述]
    
    # [第二部分标题]
    [第二部分描述]
    
    ...
    
    # 结论
    [结论描述]
    """
    
    try:
        response = await llm.complete(prompt)
        return str(response)
    except Exception as e:
        print(f"生成大纲时出错: {str(e)}")
        # 提供一个默认大纲作为备用
        return """
        # 引言
        介绍人工智能在医疗领域的发展背景和重要性。
        
        # 诊断与疾病识别
        AI如何帮助医生更准确地诊断疾病。
        
        # 医学影像分析
        AI在X光、CT、MRI等医学影像分析中的应用。
        
        # 药物研发与发现
        AI如何加速药物研发过程和降低成本。
        
        # 个性化医疗
        AI如何实现基于患者个体差异的个性化治疗方案。
        
        # 结论
        总结AI在医疗领域的影响和未来发展方向。
        """

# 将大纲拆分为部分
def split_outline_into_sections(outline):
    sections = []
    current_section = ""
    section_number = 0
    
    for line in outline.split('\n'):
        line = line.strip()
        if line.startswith('# '):
            if current_section:
                sections.append((current_section, section_number))
                section_number += 1
            current_section = line
        elif line:  # 只添加非空行
            current_section += "\n" + line
    
    # 添加最后一部分
    if current_section:
        sections.append((current_section, section_number))
    
    return sections

# 使用LLM撰写部分内容
async def write_section_with_llm(section_info, research_results):
    section, section_number = section_info
    section_title = section.splitlines()[0] if section.splitlines() else f"部分 {section_number + 1}"
    print(f"撰写部分 {section_number + 1}: {section_title}")
    
    prompt = f"""
    基于以下研究材料和大纲部分，撰写详细的内容。
    
    大纲部分:
    {section}
    
    研究材料:
    {research_results[:2000]}  # 限制字符数以避免超出上下文长度
    
    请撰写详细、信息丰富且引人入胜的内容。使用事实和数据支持你的观点。
    合理组织段落，使用适当的过渡词连接想法。内容应该流畅、连贯，非常适合目标受众阅读。
    
    输出只应包含该部分的实际内容，不需要重复标题。
    """
    
    try:
        response = await llm.complete(prompt)
        return str(response)
    except Exception as e:
        print(f"撰写部分 {section_number + 1} 时出错: {str(e)}")
        return f"这部分内容因技术原因未能生成。这里应该是关于{section_title.replace('# ', '')}的内容。"

# 编译所有部分为最终文档
def compile_sections_into_document(sections_list):
    # 按照部分编号排序
    sorted_sections = sorted(sections_list, key=lambda x: x["section_number"])
    
    # 组合文档
    document = ""
    for section_data in sorted_sections:
        section_title = section_data["section"].splitlines()[0] if section_data["section"].splitlines() else f"部分 {section_data['section_number'] + 1}"
        document += f"{section_title}\n\n{section_data['content']}\n\n"
    
    return document

# 内容生成工作流
class ContentGenerationWorkflow(Workflow):
    @step
    async def research_topic(self, ctx: Context, ev: StartEvent) -> ResearchCompleteEvent:
        # 研究主题
        topic = getattr(ev, 'topic', '人工智能在医疗领域的应用')
        content_type = getattr(ev, 'content_type', '信息性博客文章')
        audience = getattr(ev, 'audience', '对AI和医疗感兴趣的普通读者')
        word_count = getattr(ev, 'word_count', 1500)
        style = getattr(ev, 'style', '专业但易于理解')
        
        research_results = await research_topic_online(topic)
        await ctx.set("research_results", research_results)
        await ctx.set("requirements", {
            "topic": topic,
            "content_type": content_type,
            "audience": audience,
            "word_count": word_count,
            "style": style
        })
        return ResearchCompleteEvent(status="研究完成")
    
    @step
    async def generate_outline(self, ctx: Context, ev: ResearchCompleteEvent) -> OutlineCompleteEvent:
        # 生成内容大纲
        research_results = await ctx.get("research_results")
        requirements = await ctx.get("requirements")
        outline = await generate_outline_with_llm(research_results, requirements)
        await ctx.set("outline", outline)
        
        # 将大纲分解为部分
        sections = split_outline_into_sections(outline)
        await ctx.set("sections", sections)
        await ctx.set("completed_sections", [])
        
        return OutlineCompleteEvent(status="大纲生成完成")
    
    @step
    async def dispatch_writing_tasks(self, ctx: Context, ev: OutlineCompleteEvent) -> WritingCompleteEvent:
        # 启动内容撰写任务
        sections = await ctx.get("sections")
        
        # 为每个部分发送事件
        for section, section_number in sections:
            ctx.send_event(WriteSectionEvent(section=section, section_number=section_number))
        
        return WritingCompleteEvent(status="内容撰写任务已分发")
    
    @step(num_workers=3)  # 并行处理3个部分
    async def write_section(self, ctx: Context, ev: WriteSectionEvent) -> SectionCompleteEvent:
        # 撰写单个部分
        research_results = await ctx.get("research_results")
        section_content = await write_section_with_llm((ev.section, ev.section_number), research_results)
        
        # 添加一点延迟来模拟不同部分的撰写时间
        await asyncio.sleep(1 + ev.section_number * 0.5)
        
        return SectionCompleteEvent(
            section=ev.section,
            section_number=ev.section_number,
            content=section_content
        )
    
    @step
    async def collect_sections(self, ctx: Context, ev: SectionCompleteEvent) -> StopEvent | None:
        # 收集完成的部分
        completed_sections = await ctx.get("completed_sections")
        completed_sections.append({
            "section": ev.section,
            "section_number": ev.section_number,
            "content": ev.content
        })
        await ctx.set("completed_sections", completed_sections)
        
        # 检查是否所有部分都已完成
        sections = await ctx.get("sections")
        if len(completed_sections) == len(sections):
            # 所有部分都已完成，编译最终文档
            final_document = compile_sections_into_document(completed_sections)
            return StopEvent(result=final_document)
        
        # 继续等待其他部分完成
        return None

# 运行工作流
async def run_content_workflow():
    workflow = ContentGenerationWorkflow(timeout=180, verbose=True)  # 增加超时时间
    
    try:
        result = await workflow.run(
            topic="人工智能在医疗领域的应用",
            content_type="信息性博客文章",
            audience="对AI和医疗感兴趣的普通读者",
            word_count=1500,
            style="专业但易于理解"
        )
        
        # 将生成的内容保存到文件
        with open("ai_in_healthcare_article.md", "w", encoding="utf-8") as f:
            f.write(result)
        
        print("\n工作流执行完成!")
        print(f"文章已保存到 ai_in_healthcare_article.md")
        
        # 打印文章摘要
        print("\n文章摘要:")
        lines = result.split("\n")
        print("\n".join(lines[:min(15, len(lines))]) + "\n...")
        
        return result
    except Exception as e:
        print(f"运行工作流时遇到错误: {str(e)}")
        return "工作流执行失败。请检查日志获取更多信息。"

# 主函数
if __name__ == "__main__":
    # 记录开始时间
    start_time = time.time()
    
    # 设置并运行事件循环
    try:
        # 运行工作流
        result = asyncio.run(run_content_workflow())
        
        # 计算并打印总执行时间
        execution_time = time.time() - start_time
        print(f"\n总执行时间: {execution_time:.2f}秒")
    except KeyboardInterrupt:
        print("\n用户中断了工作流")
    except Exception as e:
        print(f"\n运行主程序时遇到错误: {str(e)}")

运行工作流时遇到错误: The following events are consumed but never produced: WriteSectionEvent

总执行时间: 0.00秒


这个工作流会:
使用Tavily工具研究给定主题的最新信息
根据研究结果和要求生成内容大纲
将大纲拆分为多个部分，并并行撰写各部分内容
收集所有部分并整合为一篇完整的文章
保存生成的内容到Markdown文件

## 结语
LlamaIndex的工作流功能提供了一个强大的框架，用于构建复杂的、多步骤的AI应用流程。通过自定义事件、分支逻辑、状态管理和并发执行，开发者可以构建出高度灵活和高效的应用。
无论是简单的顺序处理流程，还是复杂的并行数据处理系统，LlamaIndex工作流都能提供所需的工具和抽象，帮助开发者专注于业务逻辑而不是底层实现细节。
对于需要构建复杂AI应用流程的开发者来说，掌握LlamaIndex工作流是一项值得投资的技能，它将显著提高开发效率和应用性能。
注意：本文中的代码示例已经过实际测试，能够正常运行。但在实际应用中，你需要根据自己的需求调整工作流结构和逻辑。更多高级功能和详细文档请参考LlamaIndex官方文档。